# Testing Runs

### Imports

In [7]:
import tensorflow as tf
import numpy as np
print(tf.__version__)
import random
import math
import pickle
import Simulation.KNNSim as sim
startIndexes = pickle.load(open('Simulation/E0F1_index_transmision.pkl', 'rb'))
usablePVindextable = pickle.load(open('Simulation/E0F1_index_I1.pkl', 'rb'))
pvNames = pickle.load(open('Simulation/E0F1Names.pkl', 'rb'))

2.15.0
knn-Sim: Data loaded from Simulation/E0F1.pkl
knn-Sim: Index filled
knn-Sim: Index build
knn-Sim: READY


In [16]:
input_dim =  len(usablePVindextable)*2 + 1 # permissionbools for PVs (dependand on subsection),current PV values , current current value
output_dim = len(usablePVindextable)
input_dim
#40104 is the index for the global optimum
optimum = list(sim.states[40104][0])
baseOptimum = []

for i in usablePVindextable:
    baseOptimum.append(optimum[i])

var_I1 = [0.4656249999999993,0.17000000000000035,0.6524999999999999,0.14566250000000042,0.34001250000000066,0.19999999999999996,0.14499999999999957,0.13628749999999912,2.239959716796875,1.40057373046875,2.3402099609375,1.9005126953125]

def get_StartingPossition():
    sectionPos = []
    for i, variance in enumerate(var_I1):
        newValue = baseOptimum[i] + (random.random()-0.5) * variance
        optimum[usablePVindextable[i]] = newValue
        sectionPos.append(newValue)
    return [np.array(optimum), np.array(sectionPos)]

def modelOutToPosition(modelOut, allowedPVs, oldPosition):
    for i, change in enumerate(modelOut):
        if allowedPVs[i]:
            oldPosition[usablePVindextable[i]] += change
        modelOut[i] = oldPosition[usablePVindextable[i]]
    return [oldPosition, modelOut]

### Load Model

In [9]:
loaded_model = tf.keras.models.load_model('ModelI1SC02BothX2300_L-0.06_I-55.06_P 0.64.keras')
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (1, None, 32)             7424      
                                                                 
 lstm_1 (LSTM)               (1, None, 32)             8320      
                                                                 
 lstm_2 (LSTM)               (1, 32)                   8320      
                                                                 
 dense (Dense)               (1, 64)                   2112      
                                                                 
 dense_1 (Dense)             (1, 32)                   2080      
                                                                 
 dense_2 (Dense)             (1, 32)                   1056      
                                                                 
 dense_3 (Dense)             (1, 16)                   5

### One Run

In [19]:
# Test the trained model
position, sectionPos = get_StartingPossition()
print(sectionPos[11])
loaded_model.layers[0].reset_states()
loaded_model.layers[1].reset_states()
sim.setK(15)
allowedPVs = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]

states = []

print(f"Allowed elemnts: {allowedPVs}")
for i, isAllowed in enumerate(allowedPVs):
    if isAllowed: 
        print(pvNames[usablePVindextable[i]])
        
y = sim.get(position)
startY = y
print(f"Starting with {y}")

minY = y
for step in range(30):
            loaded_model.layers[0].reset_states()
            loaded_model.layers[1].reset_states()
            modelIn = np.reshape(np.concatenate((allowedPVs, sectionPos, [y]), axis=0), (1, 1, input_dim))
            states.append(modelIn)
            modelOut = loaded_model(modelIn)
            # We now have our next step
            position , sectionPos = modelOutToPosition(modelOut.numpy()[0], allowedPVs, position)
            # We can calculate the loss right here
            y = sim.get(position)
            print(f"Step {step+1}:  {y : .3f} overall improvement : {startY - minY: .3f} ")
            if y < minY:
                minY = y 
            

99.13852015246778
Allowed elemnts: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
I1SC02:amplitudeSetpoint
I1SC02:phaseSetpoint
Starting with 0
Step 1:  -8.873 overall improvement :  0.000 
Step 2:  -28.840 overall improvement :  8.873 
Step 3:  -51.617 overall improvement :  28.840 
Step 4:  -68.895 overall improvement :  51.617 
Step 5:  -93.921 overall improvement :  68.895 
Step 6:  -106.426 overall improvement :  93.921 
Step 7:  -123.079 overall improvement :  106.426 
Step 8:  -131.794 overall improvement :  123.079 
Step 9:  -135.732 overall improvement :  131.794 
Step 10:  -138.068 overall improvement :  135.732 
Step 11:  -186.363 overall improvement :  138.068 
Step 12:  -186.773 overall improvement :  186.363 
Step 13:  -184.622 overall improvement :  186.773 
Step 14:  -183.141 overall improvement :  186.773 
Step 15:  -179.759 overall improvement :  186.773 
Step 16:  -161.391 overall improvement :  186.773 
Step 17:  -144.937 overall improvement :  186.773 
Step 18:  -127.303 ove